In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

### **Training and Validation Data (Preprocessing)**
Normalize and augment data using ImageDataGenerator and declare the training directory and validation directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True,
                                   fill_mode="nearest")

In [ ]:
# Training folder location
train_dir = '/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/Images_Dataset/Train'

# Using TrainGenerator to label the data and set target size
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=10,
    class_mode='categorical'
)

Found 1666 images belonging to 3 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
# Validation folder location
validation_dir = '/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/Images_Dataset/Validation'

# Using flow_from_directory to label the data and set target size
val_gen = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size=10,
    class_mode='categorical'
)

Found 166 images belonging to 3 classes.


### **Build the Model using CNN Layers**

In [ ]:
# Try Model 1
def createModel():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(300, 300, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(3, activation="softmax")
  ])


  # Compile the Model
  model.compile(optimizer="adam",
                loss="categorical_crossentropy",
                metrics=["accuracy"])
  
  return model

In [ ]:
# Try Model 2 : Add regularization (l2) + increase dropout rate (to reduce overfitting)
def createModel2():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(300, 300, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.02)),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Dense(3, activation="softmax")
  ])

  # Compile the Model
  model.compile(optimizer="adam",
                loss="categorical_crossentropy",
                metrics=["accuracy"])
  
  return model

In [ ]:
# Try Model 3 : Add dropout layer and value (0.5) + declare learning rate = 0.001
def createModel3():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(300, 300, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.02)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(3, activation="softmax")
  ])

  # Compile the Model
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer,
                loss="categorical_crossentropy",
                metrics=["accuracy"])
  
  return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
model3 = createModel3()
# Train the model with early stopping
history3 = model3.fit(train_gen,
                    epochs=20,
                    validation_data=val_gen,
                    callbacks=[early_stopping])
history3

Epoch 1/20
167/167 [==============================] - 806s 5s/step - loss: 2.7944 - accuracy: 0.6849 - val_loss: 1.9435 - val_accuracy: 0.5120
Epoch 2/20
167/167 [==============================] - 786s 5s/step - loss: 1.1500 - accuracy: 0.7449 - val_loss: 2.3997 - val_accuracy: 0.5241
Epoch 3/20
167/167 [==============================] - 797s 5s/step - loss: 1.0308 - accuracy: 0.8007 - val_loss: 1.9880 - val_accuracy: 0.5904
Epoch 4/20
167/167 [==============================] - 805s 5s/step - loss: 0.9547 - accuracy: 0.8163 - val_loss: 1.3050 - val_accuracy: 0.7349
Epoch 5/20
167/167 [==============================] - 772s 5s/step - loss: 0.9412 - accuracy: 0.8013 - val_loss: 1.5293 - val_accuracy: 0.6446
Epoch 6/20
167/167 [==============================] - 763s 5s/step - loss: 0.9228 - accuracy: 0.8241 - val_loss: 1.3458 - val_accuracy: 0.7048
Epoch 7/20
167/167 [==============================] - 762s 5s/step - loss: 0.8898 - accuracy: 0.8151 - val_loss: 1.1844 - val_accuracy: 0.7108

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Custom condition for stopping based on validation accuracy
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_accuracy'] >= 0.75:
            self.model.stop_training = True

# Create an instance of the custom early stopping callback
custom_early_stopping = CustomEarlyStopping()
model3 = createModel3()

# Train the model with custom early stopping
history = model3.fit(train_gen,
                    epochs=100,
                    validation_data=val_gen,
                    callbacks=[early_stopping, custom_early_stopping])

Epoch 1/100
167/167 [==============================] - 705s 4s/step - loss: 2.8223 - accuracy: 0.6741 - val_loss: 2.2140 - val_accuracy: 0.6265
Epoch 2/100
167/167 [==============================] - 147s 879ms/step - loss: 1.2333 - accuracy: 0.7491 - val_loss: 1.5858 - val_accuracy: 0.5723
Epoch 3/100
167/167 [==============================] - 143s 856ms/step - loss: 1.1461 - accuracy: 0.7755 - val_loss: 1.4723 - val_accuracy: 0.7108
Epoch 4/100
167/167 [==============================] - 144s 862ms/step - loss: 1.1399 - accuracy: 0.7773 - val_loss: 1.1644 - val_accuracy: 0.7470
Epoch 5/100
167/167 [==============================] - 141s 836ms/step - loss: 0.9461 - accuracy: 0.7977 - val_loss: 2.4328 - val_accuracy: 0.5422
Epoch 6/100
167/167 [==============================] - 141s 848ms/step - loss: 0.9207 - accuracy: 0.8061 - val_loss: 1.5195 - val_accuracy: 0.6627
Epoch 7/100
167/167 [==============================] - 139s 836ms/step - loss: 0.9234 - accuracy: 0.8067 - val_loss: 1.35

In [ ]:
# Using learning decoy (Cosine Annealing)s
# Define a learning rate scheduler
import math

def cosine_annealing(epoch, lr):
    max_epochs = 100  # The total number of training epochs
    max_lr = 0.01  # The maximum learning rate
    min_lr = 0.001  # The minimum learning rate

    cosine_decay = 0.5 * (1 + tf.math.cos((epoch / max_epochs) * tf.constant(math.pi)))
    decayed_lr = (max_lr - min_lr) * cosine_decay + min_lr
    return decayed_lr

# Create a callback for the learning rate scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

model3 = createModel3()

# Start training the model with model.fit()
model3.fit(train_gen,
        epochs=20,
        validation_data=val_gen,
        callbacks=[lr_scheduler])


Epoch 1/20
167/167 [==============================] - 1497s 9s/step - loss: 43.1201 - accuracy: 0.6291 - val_loss: 1.3833 - val_accuracy: 0.3614 - lr: 0.0100
Epoch 2/20
167/167 [==============================] - 817s 5s/step - loss: 0.9039 - accuracy: 0.6465 - val_loss: 1.2368 - val_accuracy: 0.3614 - lr: 0.0100
Epoch 3/20
167/167 [==============================] - 841s 5s/step - loss: 0.9068 - accuracy: 0.6465 - val_loss: 1.2863 - val_accuracy: 0.3614 - lr: 0.0100


KeyboardInterrupt: ignored

In [ ]:
# Save the model (.h5)
# model1.save('/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model.h5')
model2.save('/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model3.h5')
model3.save('/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model4.h5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
# Testing folder location
test_dir = '/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/Images_Dataset/Testing'

# Using flow_from_directory to label the data and set target size
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    class_mode='categorical'
)

Found 182 images belonging to 3 classes.


In [ ]:
# Predict the classification
y_pred = model1.predict(test_gen)

6/6 [==============================] - 24s 4s/step


In [ ]:
y_pred = y_pred.argmax(axis=1)

AxisError: ignored

In [ ]:
# True labels of test dataset
y_true = test_gen.labels

array([2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0,
       2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0,
       0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2,
       2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0,
       0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0,
       0, 0, 0, 0, 1, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 1, 0,
       0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0,
       0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 0,
       0, 2, 2, 2, 2, 2])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Accuracy Scores
# Calculate accuracy
# accuracy = accuracy_score(y_true, y_pred)

# # Print accuracy
# print("Accuracy:", accuracy)

# classification report
target_names = ['lubang_jalan', 'retakan','normal']
report = classification_report(y_true, y_pred, target_names=target_names)

# Print report
print(report)

               precision    recall  f1-score   support

Heavy_damaged       0.29      0.57      0.39        51
Light_damaged       0.00      0.00      0.00        60
  No_potholes       0.38      0.42      0.40        71

     accuracy                           0.32       182
    macro avg       0.22      0.33      0.26       182
 weighted avg       0.23      0.32      0.26       182



### **Visualization**
Accuracy and val_accuracy of the model

In [ ]:
# Get the accuracy values from the training history
accuracy = history.history3['accuracy']
val_accuracy = history.history3['val_accuracy']

# Get the number of epochs
epochs = range(1, len(accuracy) + 1)

# Plot the training and validation accuracy
plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

TypeError: ignored